In [1]:
from PIL import Image
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float
import binascii
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import imutils



# Step 1: Biometric using Face Geometry

In [2]:

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

originalImage = cv2.imread('face2.png')
img = cv2.resize(originalImage, (300,300))
face_upper = []
face_bottom = []
leftEyes_upper = []
leftEyes_bottom = []
rightEyes_upper = []
rightEyes_bottom = []




def detect_faces(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords = cascade.detectMultiScale(gray_frame, 1.3, 5)
    face_upper.append(coords[0][0])
    face_upper.append(coords[0][1])
    face_bottom.append(coords[0][0] + coords[0][2])
    face_bottom.append(coords[0][1] + coords[0][3])



    print("face",coords)

    if len(coords) > 1:
        biggest = (0, 0, 0, 0)
        for i in coords:
            if i[3] > biggest[3]:
                biggest = i
        biggest = np.array([i], np.int32)
    elif len(coords) == 1:
        biggest = coords
    else:
        return None
    for (x, y, w, h) in biggest:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,255),2)    

        frame = img[y:y + h, x:x + w]
    return frame


def detect_eyes(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = cascade.detectMultiScale(gray_frame, 1.3, 5)  # detect eyes
    print("eyes" , eyes)
    
    width = np.size(img, 1)  # get face frame width
    height = np.size(img, 0)  # get face frame height
    left_eye = None
    right_eye = None
    for (x, y, w, h) in eyes:
        if y > height / 2:
            pass
        eyecenter = x + w / 2  # get the eye center

        if eyecenter < width * 0.5:
            left_eye = img[y:y + h, x:x + w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,255),2)
            leftEyes_upper.append(x)
            leftEyes_upper.append(y)
            leftEyes_bottom.append(x+w)
            leftEyes_bottom.append(y+h)
            cv2.line(img,tuple(leftEyes_upper),tuple(leftEyes_bottom), (0, 0, 255), 3)

        else:
            right_eye = img[y:y + h, x:x + w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,255),2)
            rightEyes_upper.append(x)
            rightEyes_upper.append(y)
            rightEyes_bottom.append(x+w)
            rightEyes_bottom.append(y+h)
            cv2.line(img,tuple(rightEyes_upper),tuple(rightEyes_bottom), (255, 0, 0), 3)


    return left_eye, right_eye


face_frame = detect_faces(img, face_cascade)
if face_frame is not None:
    eyes = detect_eyes(img, eye_cascade)

    
cv2.imshow('my image',img)
cv2.waitKey(0)
cv2.destroyAllWindows() 



face [[ 82  48 178 178]]
eyes [[178  91  43  43]
 [117  90  47  47]]


In [3]:
print("face_upper", face_upper)
print("face_bottom", face_bottom)

print("leftEyes_upper", leftEyes_upper)
print("leftEyes_bottom", leftEyes_bottom)

print("rightEyes_upper", rightEyes_upper)
print("rightEyes_bottom", rightEyes_bottom)




cv2.line(img, tuple(face_upper), tuple(face_bottom), (0, 255, 0), 3)
cv2.line(img, tuple(leftEyes_upper), tuple(leftEyes_bottom), (255, 0, 0), 3) 
cv2.line(img, tuple(rightEyes_upper), tuple(rightEyes_bottom), (0, 0, 255), 3) 
cv2.circle(img, tuple(leftEyes_upper), 5, (255, 0, 0),3)
cv2.circle(img, tuple(leftEyes_bottom), 5, (255, 0, 0),3) 
cv2.circle(img, tuple(rightEyes_bottom), 5, (0, 0, 255),3) 
cv2.circle(img, tuple(rightEyes_upper), 5, (0, 0, 255),3) 




cv2.imshow('my image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


face_upper [82, 48]
face_bottom [260, 226]
leftEyes_upper [117, 90]
leftEyes_bottom [164, 137]
rightEyes_upper [178, 91]
rightEyes_bottom [221, 134]


In [4]:
center_face = []
center_leftEyes = []
center_rightEyes = []
center_face.append(int((face_upper[0]+face_bottom[0])/2))
center_face.append(int((face_upper[1]+face_bottom[1])/2))

center_leftEyes.append(int((leftEyes_upper[0]+leftEyes_bottom[0])/2))
center_leftEyes.append(int((leftEyes_upper[1]+leftEyes_bottom[1])/2))

center_rightEyes.append(int((rightEyes_upper[0]+rightEyes_bottom[0])/2))
center_rightEyes.append(int((rightEyes_upper[1]+rightEyes_bottom[1])/2))

cv2.circle(img, tuple(center_face), 5, (0, 0, 0),3) 
cv2.circle(img, tuple(center_leftEyes), 5, (0, 0, 0),3) 
cv2.circle(img, tuple(center_rightEyes), 5, (0, 0, 0),3) 


cv2.imshow('my image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Step 2: Key Generation

In [5]:
distVector = []
distVector.append(np.sqrt( (center_face[0]-rightEyes_upper[0])**2 + (center_face[1]-rightEyes_upper[1])**2 ))
distVector.append(np.sqrt( (center_face[0]-rightEyes_bottom[0])**2 + (center_face[1]-rightEyes_bottom[1])**2 ))
distVector.append(np.sqrt( (center_face[0]-leftEyes_upper[0])**2 + (center_face[1]-leftEyes_upper[1])**2 ))
distVector.append(np.sqrt( (center_face[0]-leftEyes_bottom[0])**2 + (center_face[1]-leftEyes_bottom[1])**2 ))
distVector.append(np.sqrt( (center_face[0]-center_rightEyes[0])**2 + (center_face[1]-center_rightEyes[1])**2 ))
distVector.append(np.sqrt( (center_face[0]-center_leftEyes[0])**2 + (center_face[1]-center_leftEyes[1])**2 ))

distVector

[46.52956049652737,
 50.08991914547278,
 71.58910531638176,
 7.0,
 37.53664875824692,
 39.20459156782532]

In [6]:
'''
Variance and standard deviation is calculated for pairwise Euclidean distance
'''
import statistics 

variance = statistics.variance(distVector)
std_dev = statistics.stdev(distVector)
print("Variance of sample set is % s"  %(variance))
print("Standard Deviation of sample is % s " %(std_dev)) 

Variance of sample set is 442.84285130443436
Standard Deviation of sample is 21.043831668791555 


In [7]:
'''
Unique 3 digit key generation
Key= (Most Significant 2 Digits of p1) * 10 + (Least Significant 1 Digits p2)
Where
P1 = variance of Hand points pairs wise distance
P2 = standard deviation of Hand points pairs wise distance

'''
MSB = int(str(variance)[:2])
LSB = int(str(std_dev)[-1:])
#print(MSB,LSB)
key = (MSB)* 10 + (LSB)
print("Key Generated:- ",key)

Key Generated:-  445


# Step 3: Encrypt msg using Transposition Cipher

In [8]:
Number = key
Sum = 0
while(Number > 0):
    r = Number % 10
    Sum = Sum + r
    Number = Number//10
print(Sum)
face_key = Sum


13


In [9]:
#Transposition cipher 

def encryptMessage(key, message):
    ciphertext = [''] * key
   
    for col in range(key):
        position = col
        while position < len(message):
            ciphertext[col] += message[position]
            position += key
    return ''.join(ciphertext) #Cipher text
    
myMessage = input("Enter data to be encoded(Without Space) : ")
print("len",len(myMessage))
#myMessage = "Transposition Cipher"

ciphertext = encryptMessage(face_key, myMessage)
   
print("Cipher Text is")
print(ciphertext)
#pyperclip.copy(ciphertext)

Enter data to be encoded(Without Space) : Biometric Steganography lsb123
len 30
Cipher Text is
Bgbia1on2mo3egtrraipch yS tles


# # Step 4: Image Steganography using LSB


In [10]:
def to_bin(data):
    """Convert `data` to binary format as string"""
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")
        

In [11]:
'''
- First carrier image or cover image has been read and converted in to array of bits
- the secret message which is in the form of characters are converted in to the “ASCII” values
- ASCII values are converted in to array of bits
- key will gives unique pixel position of cover image to embed the data LSB of particular image pixel of cover object

Output: Stego-image has been generated which contains secret message embedded within cover image.

'''
#res = input("Enter data to be encoded(Without Space) : ") 
res= str(ciphertext)
print("res",res)

#data = ''.join(format(ord(i), 'b') for i in res) 
#print("The string after binary conversion : " + data)
data = to_bin(res)
msg_len = len(data)
print("msg length",msg_len)
            
    
i=0

with Image.open("source_img.png") as img:
   
    width, height = img.size
    print(width)
    print(height)
    #img.show()
    print(key)
    for x in range(key, width):
        for y in range(key, height):
            pixel = list(img.getpixel((x, y)))
            for n in range(0,3):
                if(i < len(data)):
                    pixel[n] = pixel[n] & ~1 | int(data[i])
                    i+=1
                    
            img.putpixel((x,y), tuple(pixel))
            
    
    img.save("source_secret.png", "PNG")
        
        

res Bgbia1on2mo3egtrraipch yS tles
msg length 240
4000
2249
445


# Step 5: Data Extraction using LSB

In [12]:
'''
- Embedded secret data has extracted from stego-image
- Extracting secret data the hand key is used

'''
extracted_bin = []
with Image.open("source_secret.png") as img:
    width, height = img.size
    byte = []
    for x in range(key, width):
        for y in range(key, height):
            pixel = list(img.getpixel((x, y)))
            for n in range(0,3):
                extracted_bin.append(pixel[n]&1)

data = "".join([str(x) for x in extracted_bin])
msg =''

for i in range(msg_len):
    msg+=str(extracted_bin[i])
#print(msg)

decoded_msg = ''.join(chr(int(msg[i*8:i*8+8],2)) for i in range(len(msg)//8))
print("decoded_msg:= ",decoded_msg)


decoded_msg:=  Bgbia1on2mo3egtrraipch yS tles


# Step 6: decrypt msg using Transposition Cipher

In [13]:
import math
def decryptMessage(key, message):
    
    numOfColumns = math.ceil(len(message) / key)
    numOfRows = key
    numOfShadedBoxes = (numOfColumns * numOfRows) - len(message)
    plaintext = [""]  * numOfColumns
    col = 0
    row = 0
   
    for symbol in message:
        plaintext[col] += symbol
        col += 1
        if (col == numOfColumns) or (col == numOfColumns - 1 and row >= numOfRows - numOfShadedBoxes):
                col = 0 
                row += 1 
    return ''.join(plaintext)
            
plaintext = decryptMessage(face_key, decoded_msg)
   
print("The plain text is")
print(plaintext)


The plain text is
Biometric Steganography lsb123


# Step 7: PSNR (Peak Signal to Noise Ratio)

In [14]:
#calculate PSNR
import math
original = cv2.imread('source_img.png')
updated = cv2.imread('source_secret.png',1)

def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d=psnr(original,updated)
print(d)

101.47341049343112


# Step 8: MSE

In [15]:
#calculate MSE
def mse(original1, contrast1):
    err = np.sum((original1.astype("float") - contrast1.astype("float")) ** 2)
    err /= float(original1.shape[0] * original1.shape[1])
    return err

MSE=mse(original,updated)
print(MSE)

1.3895064473099155e-05
